# Exploring Hacker News Posts
[Hacker News](https://news.ycombinator.com/) is a site, where user-submitted stories (known as "posts") receive votes and comments, similar to Reddit. The site is extremely popular in technology and startup circles, mainly because the posts that make it to the top of the Hacker News listings can get hundreds of thousands of visitors as a result.

The dataset can be found on [Kaggle](https://www.kaggle.com/datasets/hacker-news/hacker-news-posts). Below are the descriptions of the columns.

| <center>Column | <center>Description |
|---:|:---|
| `id` | The unique identifier from Hacker News for the post |
| `title` | The title of the post |
| `url` | The URL the posts links to, if the post has a URL |
| `num_points` | 	The number of points the post acquired, calculated as the total no. of upvotes minus total no. of downvotes |
| `num_comments` | The number of comments on the post |
| `author` | The username of the person who submitted the post |
| `created_at` | The date and time of the post's submission |

In [1]:
#This code block is to open, read and convert the file into a list of lists. Also print the first rows to check the result
import csv
open_file = open("C:/Users/Daniel Mendes/Projects/Databases/HN_Posts.csv", encoding="utf8")
read_file = csv.reader(open_file)
hn_dataset = list(read_file)
hn_headers = hn_dataset[0]
hn_dataset = hn_dataset[1:]
print(hn_dataset[:2])
print("\n")
print("Length: {:,}".format(len(hn_dataset)))

[['12579008', 'You have two days to comment if you want stem cells to be classified as your own', 'http://www.regulations.gov/document?D=FDA-2015-D-3719-0018', '1', '0', 'altstar', '9/26/2016 3:26'], ['12579005', 'SQLAR  the SQLite Archiver', 'https://www.sqlite.org/sqlar/doc/trunk/README.md', '1', '0', 'blacksqr', '9/26/2016 3:24']]


Length: 293,119


In [2]:
#Removing the rows of posts w/o comments. Printet the length to check how many posts is removed
hn_dataset_final = []
for row in hn_dataset:
    comments = int(row[4])
    if comments != 0:
        hn_dataset_final.append(row)
        
print(hn_dataset_final[:2])
print("\n")
print("Length: {:,}".format(len(hn_dataset_final)))

[['12578975', 'Saving the Hassle of Shopping', 'https://blog.menswr.com/2016/09/07/whats-new-with-your-style-feed/', '1', '1', 'bdoux', '9/26/2016 3:13'], ['12578908', 'Ask HN: What TLD do you use for local development?', '', '4', '7', 'Sevrene', '9/26/2016 2:53']]


Length: 80,401


***
The posts on Hacker News with titles **Ask HN** or **Show HN** means:
 - **Ask HN** posts to ask the Hacker News community a specific question.
 - **Show HN** posts to show the Hacker News community a project, product, or something interesting.
***
We are interested in the above titles, **Ask HN** or **Show HN**. By using these two types of posts, we will analyze

1. Do **Ask HN** or **Show HN** recevie more comments on average?
2. Do posts created at a certain time receive more comments on average?
***

In [3]:
ask_posts = []
show_posts = []
other_posts = []
for row in hn_dataset_final:
    title = row[1]
    if title.lower().startswith('ask hn') == True:
        ask_posts.append(row)
    elif title.lower().startswith('show hn') == True:
        show_posts.append(row)
    else:
        other_posts.append(row)
print("{} length is {:,} and {} length is {:,}. We also have {:,} other posts.".format("Ask Posts",len(ask_posts),"Show Posts",len(show_posts),len(other_posts)))

Ask Posts length is 6,911 and Show Posts length is 5,059. We also have 68,431 other posts.


On the previous code, we separated the ask posts and the show posts into two lists of lists named `ask_posts` and `show_posts`.
Next, let's determine if ask posts or show posts receive more comments on average.

In [4]:
ask_comments_qtd = 0
ask_comments_sum = 0
for row in ask_posts:
    comments = int(row[4])
    ask_comments_sum += comments
    ask_comments_qtd += 1

show_comments_qtd = 0
show_comments_sum = 0
for row in show_posts:
    comments = int(row[4])
    show_comments_sum += comments
    show_comments_qtd += 1

ask_comments_avg = round(ask_comments_sum / ask_comments_qtd,2)
show_comments_avg = round(show_comments_sum / ask_comments_qtd,2)

print("In this website, asks posts have {} comments in average, againist {} for show posts.".format(ask_comments_avg,show_comments_avg))

In this website, asks posts have 13.74 comments in average, againist 7.18 for show posts.


On the previous code, we saw that, on average, ask posts receive more comments than show posts. Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

Next, we'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

1. Calculate the number of ask posts created in each hour of the day, along with the number of comments received.
2. Calculate the average number of comments ask posts receive by hour created.

In [5]:
#creating a new database based in ask posts, only with time and comments quantity
import datetime as dt
result_list = []
for row in ask_posts:
    result_list.append([row[6],int(row[4])])
print(result_list[:3])

[['9/26/2016 2:53', 7], ['9/26/2016 1:17', 3], ['9/25/2016 22:48', 3]]


In [6]:
counts_by_hour = {}
comments_by_hour = {}
for row in result_list:
    date = row[0]
    date = dt.datetime.strptime(date,"%m/%d/%Y %H:%M")
    row[0] = date

for row in result_list:
    date = row[0]
    comments = int(row[1])
    date = dt.datetime.strftime(date, "%H")
    if date not in counts_by_hour:
        counts_by_hour[date] = 1
        comments_by_hour[date] = comments
    else:
        counts_by_hour[date] += 1
        comments_by_hour[date] += comments
        
print(counts_by_hour)
print(comments_by_hour)

{'02': 227, '01': 223, '22': 287, '21': 407, '19': 420, '17': 404, '15': 467, '14': 378, '13': 326, '11': 251, '10': 219, '09': 176, '07': 157, '03': 212, '16': 415, '08': 190, '00': 231, '23': 276, '20': 392, '18': 452, '12': 274, '04': 186, '06': 176, '05': 165}
{'02': 2996, '01': 2089, '22': 3372, '21': 4500, '19': 3954, '17': 5547, '15': 18525, '14': 4972, '13': 7245, '11': 2797, '10': 3013, '09': 1477, '07': 1585, '03': 2154, '16': 4466, '08': 2362, '00': 2277, '23': 2297, '20': 4462, '18': 4877, '12': 4234, '04': 2360, '06': 1587, '05': 1838}


In [7]:
avg_comments_by_hour = []
for hour in counts_by_hour:
    average = comments_by_hour[hour]/counts_by_hour[hour]
    avg_comments_by_hour.append([hour, round(average,2)])
print(avg_comments_by_hour)

[['02', 13.2], ['01', 9.37], ['22', 11.75], ['21', 11.06], ['19', 9.41], ['17', 13.73], ['15', 39.67], ['14', 13.15], ['13', 22.22], ['11', 11.14], ['10', 13.76], ['09', 8.39], ['07', 10.1], ['03', 10.16], ['16', 10.76], ['08', 12.43], ['00', 9.86], ['23', 8.32], ['20', 11.38], ['18', 10.79], ['12', 15.45], ['04', 12.69], ['06', 9.02], ['05', 11.14]]


In [8]:
swap_avg_by_hour = []
for row in avg_comments_by_hour:
    hour = row[0]
    average = row[1]
    swap_avg_by_hour.append([average,hour])
print(swap_avg_by_hour)

[[13.2, '02'], [9.37, '01'], [11.75, '22'], [11.06, '21'], [9.41, '19'], [13.73, '17'], [39.67, '15'], [13.15, '14'], [22.22, '13'], [11.14, '11'], [13.76, '10'], [8.39, '09'], [10.1, '07'], [10.16, '03'], [10.76, '16'], [12.43, '08'], [9.86, '00'], [8.32, '23'], [11.38, '20'], [10.79, '18'], [15.45, '12'], [12.69, '04'], [9.02, '06'], [11.14, '05']]


In [9]:
sorted_swap_list = sorted(swap_avg_by_hour,reverse=True)
print("Top 5 Hours for Ask Posts Comments \n")
for row in sorted_swap_list[:4]:
    print("{}hrs with {} in average comments per post".format(row[1],row[0]))


Top 5 Hours for Ask Posts Comments 

15hrs with 39.67 in average comments per post
13hrs with 22.22 in average comments per post
12hrs with 15.45 in average comments per post
10hrs with 13.76 in average comments per post
